# Helper Functions

This notebook contains the code to build and test the helper functions used to clean the data. The functions are built so they can be used by any team member in any notebook.

In [1]:
# import libraries
import sqlite3
import pandas as pd
import regex as re

## Load data

In [2]:
# movie gross data
movie_gross = pd.read_csv('../Data/bom.movie_gross.csv')
movie_gross.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [3]:
# connect to sql db
conn= sqlite3.connect("../Data/im.db")

In [4]:
#movi_basics table
q = """
SELECT *
FROM movie_basics
"""
movie_basics_df=pd.read_sql(q, conn)
movie_basics_df.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [5]:
#movie_ratings table
q = """
SELECT *
FROM movie_ratings
"""
movie_ratings_df=pd.read_sql(q, conn)
movie_ratings_df.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [6]:
#movie_akas table
q = """
SELECT * FROM movie_akas
"""
movie_akas_df = pd.read_sql(q, conn)
movie_akas_df.head()

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0


In [7]:
# movie budget data
movie_budgets = pd.read_csv('../Data/tn.movie_budgets.csv')
movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


## Combine tables from sql db

In [8]:
# Joinings movie basics, movie ratings, and movie_akas tables on IDs
df = movie_ratings_df.merge(movie_basics_df, how='inner', on='movie_id')
df= df.merge(movie_akas_df, how='inner', on = "movie_id")
df.head()

,movie_id,averagerating,numvotes,primary_title,original_title,start_year,runtime_minutes,genres,ordering,title,region,language,types,attributes,is_original_title
0,tt1042974,6.4,20,Just Inès,Just Inès,2010,90.0,Drama,1,Just Inès,None,None,original,None,1.0
1,tt1042974,6.4,20,Just Inès,Just Inès,2010,90.0,Drama,2,Samo Ines,RS,None,imdbDisplay,None,0.0
2,tt1042974,6.4,20,Just Inès,Just Inès,2010,90.0,Drama,3,Just Inès,GB,None,None,None,0.0
3,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",10,The Legend of Hercules,None,None,original,None,1.0
4,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",11,Hércules - A Lenda Começa,PT,None,imdbDisplay,None,0.0


In [9]:
# view all region options
df['region'].unique()

array([None, 'RS', 'GB', 'PT', 'JP', 'AL', 'HR', 'VN', 'IT', 'GR', 'UA',
       'RO', 'US', 'CA', 'CL', 'MX', 'RU', 'TR', 'IL', 'AR', 'PE', 'FR',
       'ES', 'BR', 'PL', 'BG', 'HU', 'VE', 'SE', 'XWW', 'XEU', 'SI', 'UY',
       'FI', 'DE', 'PH', 'NO', 'DK', 'EE', 'BE', 'LT', 'CO', 'CZ', 'AT',
       'GE', 'IE', 'ZA', 'BA', 'LV', 'AU', 'LU', 'IN', 'KR', 'IR', 'NL',
       'DZ', 'CN', 'NZ', 'ID', 'MY', 'JO', 'RW', 'EG', 'CH', 'LI', 'TH',
       'BF', 'CSXX', 'SG', 'NP', 'SK', 'BD', 'PA', 'KE', 'PK', 'PS', 'HK',
       'MK', 'KZ', 'TW', 'LK', 'IS', 'MW', 'DO', 'BO', 'PY', 'SL', 'AZ',
       'AF', 'AE', 'PR', 'CU', 'NG', 'IQ', 'AM', 'CY', 'GU', 'HN', 'XAS',
       'TN', 'BY', 'SA', 'HT', 'MT', 'BH', 'KH', 'KG', 'FO', 'SV', 'XSA',
       'GT', 'LB', 'BT', 'CR', 'XKV', 'UZ', 'MA', 'ME', 'LA', 'SB', 'XKO',
       'SY', 'TZ', 'QA', 'MN', 'EC', 'JM', 'NE', 'AN', 'MV', 'TT', 'ET',
       'CF', 'CM', 'MM', 'TJ', 'SM', 'MD', 'GW', 'KW', 'AD', 'BB', 'MZ',
       'BS', 'SO', 'BZ', 'IM', 'MC', 'AW', 

In [10]:
# keep only US
us_movies = df[df['region']=='US']
us_movies.head()

,movie_id,averagerating,numvotes,primary_title,original_title,start_year,runtime_minutes,genres,ordering,title,region,language,types,attributes,is_original_title
14,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",20,The Legend of Hercules,US,None,None,None,0.0
26,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",31,Untitled Hercules Project,US,None,working,None,0.0
29,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",5,Hercules: The Legend Begins,US,None,working,None,0.0
42,tt1094666,7.0,1613,The Hammer,Hamill,2010,108.0,"Biography,Drama,Sport",4,Hamill,US,None,festival,None,0.0
45,tt1094666,7.0,1613,The Hammer,Hamill,2010,108.0,"Biography,Drama,Sport",7,The Hammer,US,None,imdbDisplay,None,0.0


In [11]:
#dropping duplicate movie_id's
us_movies= us_movies.drop_duplicates(subset=['movie_id'])
us_movies.head()

,movie_id,averagerating,numvotes,primary_title,original_title,start_year,runtime_minutes,genres,ordering,title,region,language,types,attributes,is_original_title
14,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",20,The Legend of Hercules,US,None,None,None,0.0
42,tt1094666,7.0,1613,The Hammer,Hamill,2010,108.0,"Biography,Drama,Sport",4,Hamill,US,None,festival,None,0.0
64,tt1161457,4.2,148,Vanquisher,The Vanquisher,2016,90.0,"Action,Adventure,Sci-Fi",1,Vanquisher,US,None,None,new title,0.0
76,tt1171222,5.1,8296,Baggage Claim,Baggage Claim,2013,96.0,Comedy,5,Baggage Claim,US,None,None,None,0.0
85,tt1174693,5.8,2381,The Four-Faced Liar,The Four-Faced Liar,2010,87.0,"Comedy,Drama,Romance",5,The Four-Faced Liar,US,None,None,None,0.0


## Adding in gross numbers to the table

#### Clean title and year info to merge
-rules: 
1. all lower
2. only alphanumeric
3. remove spaces
    *to eliminate issues with titles contaning "Star Wars:Episode 3" vs "Star Wars: Episode 3" and "Mc'Donald" vs "mc donald"
4. use primary title as title

*merging on title AND year to show difference in the movies with the same name (remakes)

In [12]:
# use RegEx to keep only alpha-numeric values, remove spaces and make all letters lowercase
us_movies["title"] = us_movies["title"].map(lambda x: re.sub(r'[^A-Za-z0-9]+', '', x).lower())
movie_budgets["movie"] = movie_budgets["movie"].map(lambda x: re.sub(r'[^A-Za-z0-9]+', '', x).lower())
movie_gross["title"] = movie_gross["title"].map(lambda x: re.sub(r'[^A-Za-z0-9]+', '', x).lower())

In [13]:
# rename columns for clean merging
movie_budgets.rename(columns={"movie": "title", "domestic_gross": "domestic_gross_movie_budgets",
                              "worldwide_gross": "worldwide_gross_movie_budgets"}, inplace=True)

movie_gross.rename(columns={"domestic_gross": "domestic_gross_movie_gross", 
                            "foreign_gross": "foreign_gross_movie_gross"}, inplace=True)

us_movies.rename(columns={"start_year": "year"}, inplace=True)

In [14]:
# add a year column for merging on year
movie_budgets["year"] = movie_budgets["release_date"].map(lambda x: int(x[-4:]))
movie_budgets.head()

,id,release_date,title,production_budget,domestic_gross_movie_budgets,worldwide_gross_movie_budgets,year
0,1,"Dec 18, 2009",avatar,"$425,000,000","$760,507,625","$2,776,345,279",2009
1,2,"May 20, 2011",piratesofthecaribbeanonstrangertides,"$410,600,000","$241,063,875","$1,045,663,875",2011
2,3,"Jun 7, 2019",darkphoenix,"$350,000,000","$42,762,350","$149,762,350",2019
3,4,"May 1, 2015",avengersageofultron,"$330,600,000","$459,005,868","$1,403,013,963",2015
4,5,"Dec 15, 2017",starwarsepviiithelastjedi,"$317,000,000","$620,181,382","$1,316,721,747",2017


In [15]:
movie_gross.head()

,title,studio,domestic_gross_movie_gross,foreign_gross_movie_gross,year
0,toystory3,BV,415000000.0,652000000,2010
1,aliceinwonderland2010,BV,334200000.0,691300000,2010
2,harrypotterandthedeathlyhallowspart1,WB,296000000.0,664300000,2010
3,inception,WB,292600000.0,535700000,2010
4,shrekforeverafter,P/DW,238700000.0,513900000,2010


In [16]:
#combine tables on title and year columns to distinguish different movies with same name and different release dates
final_gross_df1 = us_movies.merge(movie_budgets, on=["title", "year"], how="inner")
final_gross_df1

,movie_id,averagerating,numvotes,primary_title,original_title,year,runtime_minutes,genres,ordering,title,region,language,types,attributes,is_original_title,id,release_date,production_budget,domestic_gross_movie_budgets,worldwide_gross_movie_budgets
0,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",20,thelegendofhercules,US,None,None,None,0.0,42,"Jan 10, 2014","$70,000,000","$18,848,538","$58,953,319"
1,tt1171222,5.1,8296,Baggage Claim,Baggage Claim,2013,96.0,Comedy,5,baggageclaim,US,None,None,None,0.0,38,"Sep 27, 2013","$8,500,000","$21,569,509","$22,885,836"
2,tt1210166,7.6,326657,Moneyball,Moneyball,2011,133.0,"Biography,Drama,Sport",14,moneyball,US,None,imdbDisplay,None,0.0,15,"Sep 23, 2011","$50,000,000","$75,605,492","$111,300,835"
3,tt1212419,6.5,87288,Hereafter,Hereafter,2010,129.0,"Drama,Fantasy,Romance",4,hereafter,US,None,None,None,0.0,61,"Oct 15, 2010","$50,000,000","$32,746,941","$108,660,270"
4,tt1232829,7.2,477771,21 Jump Street,21 Jump Street,2012,109.0,"Action,Comedy,Crime",26,21jumpstreet,US,None,imdbDisplay,None,0.0,44,"Mar 16, 2012","$42,000,000","$138,447,667","$202,812,429"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067,tt2333784,6.1,156432,The Expendables 3,The Expendables 3,2014,126.0,"Action,Adventure,Thriller",11,theexpendables3,US,None,None,None,0.0,96,"Aug 15, 2014","$100,000,000","$39,322,544","$209,461,378"
1068,tt2334871,4.5,29997,Snatched,Snatched,2017,90.0,"Action,Adventure,Comedy",12,snatched,US,None,None,None,0.0,51,"May 12, 2017","$42,000,000","$45,852,178","$57,852,177"
1069,tt2386490,7.6,60769,How to Train Your Dragon: The Hidden World,How to Train Your Dragon: The Hidden World,2019,104.0,"Action,Adventure,Animation",20,howtotrainyourdragonthehiddenworld,US,None,None,None,0.0,56,"Feb 22, 2019","$129,000,000","$160,791,800","$519,258,283"
1070,tt2704998,7.0,163279,Game Night,Game Night,2018,100.0,"Action,Comedy,Crime",15,gamenight,US,None,imdbDisplay,None,0.0,94,"Feb 23, 2018","$37,000,000","$69,179,066","$117,378,084"


In [17]:
final_gross_df2=us_movies.merge(movie_gross, on=['title', 'year'], how='inner').drop_duplicates(subset=['movie_id'])
final_gross_df2

,movie_id,averagerating,numvotes,primary_title,original_title,year,runtime_minutes,genres,ordering,title,region,language,types,attributes,is_original_title,studio,domestic_gross_movie_gross,foreign_gross_movie_gross
0,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",20,thelegendofhercules,US,None,None,None,0.0,LG/S,18800000.0,42400000
1,tt1171222,5.1,8296,Baggage Claim,Baggage Claim,2013,96.0,Comedy,5,baggageclaim,US,None,None,None,0.0,FoxS,21600000.0,887000
2,tt1210166,7.6,326657,Moneyball,Moneyball,2011,133.0,"Biography,Drama,Sport",14,moneyball,US,None,imdbDisplay,None,0.0,Sony,75600000.0,34600000
3,tt1212419,6.5,87288,Hereafter,Hereafter,2010,129.0,"Drama,Fantasy,Romance",4,hereafter,US,None,None,None,0.0,WB,32700000.0,72500000
4,tt1232829,7.2,477771,21 Jump Street,21 Jump Street,2012,109.0,"Action,Comedy,Crime",26,21jumpstreet,US,None,imdbDisplay,None,0.0,Sony,138400000.0,63100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,tt2333784,6.1,156432,The Expendables 3,The Expendables 3,2014,126.0,"Action,Adventure,Thriller",11,theexpendables3,US,None,None,None,0.0,LGF,39300000.0,175300000
1058,tt2334871,4.5,29997,Snatched,Snatched,2017,90.0,"Action,Adventure,Comedy",12,snatched,US,None,None,None,0.0,Fox,45900000.0,15000000
1059,tt2463288,6.0,48134,Walk of Shame,Walk of Shame,2014,95.0,Comedy,12,walkofshame,US,None,None,None,0.0,FCW,59200.0,NaN
1060,tt2704998,7.0,163279,Game Night,Game Night,2018,100.0,"Action,Comedy,Crime",15,gamenight,US,None,imdbDisplay,None,0.0,WB (NL),69200000.0,48500000


In [18]:
#shows all gross tables merged
final_gross_all = pd.concat([final_gross_df1, final_gross_df2]).drop_duplicates(subset=['movie_id'])
final_gross_all

,movie_id,averagerating,numvotes,primary_title,original_title,year,runtime_minutes,genres,ordering,title,...,attributes,is_original_title,id,release_date,production_budget,domestic_gross_movie_budgets,worldwide_gross_movie_budgets,studio,domestic_gross_movie_gross,foreign_gross_movie_gross
0,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",20,thelegendofhercules,...,None,0.0,42.0,"Jan 10, 2014","$70,000,000","$18,848,538","$58,953,319",NaN,NaN,NaN
1,tt1171222,5.1,8296,Baggage Claim,Baggage Claim,2013,96.0,Comedy,5,baggageclaim,...,None,0.0,38.0,"Sep 27, 2013","$8,500,000","$21,569,509","$22,885,836",NaN,NaN,NaN
2,tt1210166,7.6,326657,Moneyball,Moneyball,2011,133.0,"Biography,Drama,Sport",14,moneyball,...,None,0.0,15.0,"Sep 23, 2011","$50,000,000","$75,605,492","$111,300,835",NaN,NaN,NaN
3,tt1212419,6.5,87288,Hereafter,Hereafter,2010,129.0,"Drama,Fantasy,Romance",4,hereafter,...,None,0.0,61.0,"Oct 15, 2010","$50,000,000","$32,746,941","$108,660,270",NaN,NaN,NaN
4,tt1232829,7.2,477771,21 Jump Street,21 Jump Street,2012,109.0,"Action,Comedy,Crime",26,21jumpstreet,...,None,0.0,44.0,"Mar 16, 2012","$42,000,000","$138,447,667","$202,812,429",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050,tt0975645,6.8,68315,Hitchcock,Hitchcock,2012,98.0,"Biography,Comedy,Drama",12,hitchcock,...,new title,0.0,NaN,NaN,NaN,NaN,NaN,FoxS,6000000.0,17600000
1054,tt2055765,5.8,11145,The English Teacher,The English Teacher,2013,93.0,"Comedy,Drama",19,theenglishteacher,...,None,0.0,NaN,NaN,NaN,NaN,NaN,Cdgm.,105000.0,NaN
1056,tt2132285,5.6,77726,The Bling Ring,The Bling Ring,2013,90.0,"Biography,Crime,Drama",1,theblingring,...,new title,0.0,NaN,NaN,NaN,NaN,NaN,A24,5800000.0,13300000
1059,tt2463288,6.0,48134,Walk of Shame,Walk of Shame,2014,95.0,Comedy,12,walkofshame,...,None,0.0,NaN,NaN,NaN,NaN,NaN,FCW,59200.0,NaN


## Clean financial values

In [19]:
# check data types
final_gross_all[["domestic_gross_movie_budgets", "worldwide_gross_movie_budgets", "domestic_gross_movie_gross", "foreign_gross_movie_gross"]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1389 entries, 0 to 1061
Data columns (total 4 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   domestic_gross_movie_budgets   1072 non-null   object 
 1   worldwide_gross_movie_budgets  1072 non-null   object 
 2   domestic_gross_movie_gross     313 non-null    float64
 3   foreign_gross_movie_gross      147 non-null    object 
dtypes: float64(1), object(3)
memory usage: 54.3+ KB


In [20]:
# convert strings to floats
final_gross_all["domestic_gross_movie_budgets"] = final_gross_all["domestic_gross_movie_budgets"].replace('[$,()]', '', regex=True).astype(float)
final_gross_all["worldwide_gross_movie_budgets"] = final_gross_all["worldwide_gross_movie_budgets"].replace('[$,()]', '', regex=True).astype(float)
final_gross_all["foreign_gross_movie_gross"] = final_gross_all["foreign_gross_movie_gross"].replace('[$,()]', '', regex=True).astype(float)
final_gross_all["production_budget"] = final_gross_all["production_budget"].replace('[$,()]', '', regex=True).astype(float)

In [21]:
# replace missing gross values from bugets data with gross data from gross table
final_gross_all["clean_domestic_gross"] = final_gross_all["domestic_gross_movie_budgets"].fillna(final_gross_all["domestic_gross_movie_gross"])
final_gross_all["clean_worldwide_gross"] = final_gross_all["worldwide_gross_movie_budgets"].fillna(final_gross_all["domestic_gross_movie_gross"])

In [22]:
final_gross_all[["clean_domestic_gross", "clean_worldwide_gross"]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1389 entries, 0 to 1061
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   clean_domestic_gross   1385 non-null   float64
 1   clean_worldwide_gross  1385 non-null   float64
dtypes: float64(2)
memory usage: 32.6 KB


In [23]:
# drop the remaining null values
final_gross_all.dropna(subset=["clean_domestic_gross", "clean_worldwide_gross"], inplace=True)
final_gross_all[["clean_domestic_gross", "clean_worldwide_gross"]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1385 entries, 0 to 1061
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   clean_domestic_gross   1385 non-null   float64
 1   clean_worldwide_gross  1385 non-null   float64
dtypes: float64(2)
memory usage: 32.5 KB


## Investigate Rotten Tomatoes Data

Joining director and release date

In [24]:
#putting director's name and movie name in one table
q = """
SELECT movie_basics.movie_id, movie_basics.primary_title, persons.primary_name
FROM directors
JOIN movie_basics on directors.movie_id = movie_basics.movie_id
JOIN persons ON directors.person_id = persons.person_id
"""
director_to_movies = pd.read_sql(q, conn).drop_duplicates(subset=['primary_name','primary_title'])
director_to_movies

,movie_id,primary_title,primary_name
0,tt0285252,Life's a Beach,Tony Vitale
1,tt0462036,Steve Phoenix: The Untold Story,Bill Haley
2,tt0835418,The Babymakers,Jay Chandrasekhar
4,tt0878654,Bulletface,Albert Pyun
5,tt0878654,Bulletface,Joe Baile
...,...,...,...
291166,tt8999974,Madre Luna,Daysi Burbano
291167,tt9001390,The woman and the river,Bernard Lessa
291168,tt9001494,Stone Mountain Georgia 08.28.18,Tate Nova
291169,tt9001494,Stone Mountain Georgia 08.28.18,Courtney Faye Powell


rt_move_info and rt_movie_reviews

In [25]:
#load rotten tomatoes
rt_movie_info = pd.read_table('../Data/rt.movie_info.tsv')
rt_movie_info.head()


,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


# *****need to unpack rows with multiple directors

In [26]:
for index, row in rt_movie_info.iterrows():
    print(row["director"])

William Friedkin
David Cronenberg
Allison Anders
Barry Levinson
Rodney Bennett
Jay Russell
Jake Kasdan
Ray Lawrence
Taylor Hackford
Frank Marshall
nan
nan
nan
William Friedkin
Peter Baldwin
George Hickenlooper
nan
Rick Rosenthal
Carl Erik Rinsch
Jim Jarmusch
nan
nan
Martyn Burke
Keith Gordon
John Woo
Michael Polish
Roy Ward Baker
David Arquette
John Sayles
Ken Loach
nan
William Wellman
Pauly Shore
Steve Boyum
Peter Cattaneo
Edward Dmytryk
Andy Sidaris
Ivan Nagy
Terence Young
Jon Turteltaub
nan
Paolo Sorrentino
Henning Schellerup
Allen Hughes|Albert Hughes
nan
Woody Allen
Yimou Zhang
Bill Froehlich
James Wong
nan
Pat Proft
Kevin Lima
Niels Arden Oplev
Matt Bettinelli-Olpin|Tyler Gillett
Anatole Litvak
Ira Miller
Steven Spielberg
Alan Alda
Otto Preminger
Terry Jones
Patrick Lussier
Tom Hanks
James Gray
Werner Herzog
Richard Kelly
Roy Rowland
nan
Ted Tetzlaff
William Beaudine
Bruce Beresford
nan
nan
Herbert Wilcox
John Gilling
Ernst Lubitsch
Cheryl Dunye
Ben Younger
Richard Linklater
Alan

In [27]:
x = rt_movie_info.dropna(subset=["director"])
x[x["director"].str.contains("\|")]

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
43,56,Albert Hughes and his brother Allen Hughes fol...,R,Drama,Allen Hughes|Albert Hughes,Michael Henry Brown|Albert Hughes|Allen Hughes,"Oct 6, 1995","May 19, 1998",NaN,NaN,119 minutes,NaN
53,67,"After a mysterious, lost night on their honeym...",R,Horror,Matt Bettinelli-Olpin|Tyler Gillett,Lindsay Devlin,"Jan 17, 2014","Apr 29, 2014",NaN,NaN,89 minutes,20th Century Fox
100,128,NaN,NR,Art House and International,Ye Ying|Zhang Li,NaN,NaN,NaN,NaN,NaN,120 minutes,NaN
121,155,"Taking place over the course of one night, The...",R,Comedy|Drama,Jennifer Jason Leigh|Alan Cumming,Jennifer Jason Leigh|Alan Cumming,"Jun 8, 2001","Jan 15, 2002",$,"2,974,308",115 minutes,Fine Line Features
146,191,Award-winning filmmaker Rakeysh Omprakash Mehr...,NR,Drama,Rakeysh Omprakash Mehra|Rakeysh Omprakesh Mehra,Prasoon Joshi,"Jul 12, 2013","Jan 12, 2015",$,"1,626,289",188 minutes,Reliance Entertainment
...,...,...,...,...,...,...,...,...,...,...,...,...
1490,1918,A collection of 15 chapters from Tarzan and th...,NR,Action and Adventure|Classics|Documentary,Henry MacRae|Harry McRae,William Lord Wright|Ian McCloskey Heath,"Oct 1, 1929","Oct 24, 2006",NaN,NaN,290 minutes,NaN
1511,1945,"Left on a nun's doorstep, Larry, Curly and Moe...",PG,Comedy,Bobby Farrelly|Peter Farrelly,Bobby Farrelly|Peter Farrelly|Mike Cerrone,"Apr 13, 2012","Jul 17, 2012",$,"41,800,000",92 minutes,20th Century Fox
1518,1953,A glimpse into the comedic process and private...,R,Comedy|Documentary|Television,Ricki Stern|Anne Sundberg,Ricki Stern,"Jun 11, 2010","Dec 14, 2010",$,"2,927,972",84 minutes,IFC Films
1529,1966,Two rabid Boston B-ball fans go over the edge ...,PG-13,Comedy,Tom DeCerchio|Tom De Cerchio,Judd Apatow|Colin Quinn,"Apr 19, 1996","Sep 3, 2002",NaN,NaN,90 minutes,NaN


In [28]:
# directors = rt_movie_info['director']
# rt_movie_info['director'] = directors.str.split(pat="|")

In [29]:
# testing unpacking the directors
direcs = []
ids = []

for index, row in rt_movie_info.iterrows():
    try:
        for person in row["director"]:
            direcs.append(person)
            ids.append(row["id"])
    except:
        direcs.append("-")
        ids.append(row["id"])
        
y = pd.DataFrame({"rt_id": ids, "director": direcs})
y[y["rt_id"].duplicated(keep=False)]

,rt_id,director
0,1,W
1,1,i
2,1,l
3,1,l
4,1,i
...,...,...
19516,1999,E
19517,1999,v
19518,1999,a
19519,1999,n


Splitting the theatre_date column into just year. Done to be able to merge rt_movie_info with total_movie_info

In [30]:
#drop missing values
rt_movie_info.dropna(subset=['theater_date'],inplace=True)
#split theatre_date column
rt_movie_info['year']= rt_movie_info['theater_date'].map(lambda x: int(x[-4:]))
rt_movie_info.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio,year
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN,1971
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One,2012
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN,1996
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN,1994
5,8,The year is 1942. As the Allies unite overseas...,PG,Drama|Kids and Family,Jay Russell,Gail Gilchriest,"Mar 3, 2000","Jul 11, 2000",NaN,NaN,95 minutes,Warner Bros. Pictures,2000


Plan: replace person ID with direct name and movie_id with the movie name. Then we will merge RT data with the gross table data. Merge on director and release date

## Merging the rotten tomatoes table with the director movie table

##### This will allow us to match the director with the release date


The goal in this is to match the director with the movie--- hopefully each director has only released one movie on one day

In [31]:
#merging direct_to_movies with final_gross_all
total_movie_info= final_gross_all.merge(director_to_movies[['movie_id','primary_name']], on="movie_id", how="left")
total_movie_info[total_movie_info["movie_id"].duplicated(keep=False)]
#checked to make sure duplicates all had different directors

,movie_id,averagerating,numvotes,primary_title,original_title,year,runtime_minutes,genres,ordering,title,...,release_date,production_budget,domestic_gross_movie_budgets,worldwide_gross_movie_budgets,studio,domestic_gross_movie_gross,foreign_gross_movie_gross,clean_domestic_gross,clean_worldwide_gross,primary_name
4,tt1232829,7.2,477771,21 Jump Street,21 Jump Street,2012,109.0,"Action,Comedy,Crime",26,21jumpstreet,...,"Mar 16, 2012",42000000.0,138447667.0,202812429.0,NaN,NaN,NaN,138447667.0,202812429.0,Christopher Miller
5,tt1232829,7.2,477771,21 Jump Street,21 Jump Street,2012,109.0,"Action,Comedy,Crime",26,21jumpstreet,...,"Mar 16, 2012",42000000.0,138447667.0,202812429.0,NaN,NaN,NaN,138447667.0,202812429.0,Phil Lord
16,tt0985694,6.6,182320,Machete,Machete,2010,105.0,"Action,Crime,Thriller",12,machete,...,"Sep 3, 2010",12500000.0,26593646.0,46370970.0,NaN,NaN,NaN,26593646.0,46370970.0,Robert Rodriguez
17,tt0985694,6.6,182320,Machete,Machete,2010,105.0,"Action,Crime,Thriller",12,machete,...,"Sep 3, 2010",12500000.0,26593646.0,46370970.0,NaN,NaN,NaN,26593646.0,46370970.0,Ethan Maniquis
21,tt1232775,6.4,483,Drones,Drones,2010,98.0,Comedy,1,drones,...,"Dec 31, 2010",500000.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,Amber Benson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1490,tt2349144,6.4,17206,Mississippi Grind,Mississippi Grind,2015,108.0,"Comedy,Drama",11,mississippigrind,...,NaN,NaN,NaN,NaN,A24,131000.0,NaN,131000.0,131000.0,Ryan Fleck
1500,tt4337072,6.7,3003,Ghost in the Shell: The New Movie,Kôkaku Kidôtai,2015,100.0,"Action,Animation,Sci-Fi",11,ghostintheshellthenewmovie,...,NaN,NaN,NaN,NaN,FUN,101000.0,NaN,101000.0,101000.0,Kazuchika Kise
1501,tt4337072,6.7,3003,Ghost in the Shell: The New Movie,Kôkaku Kidôtai,2015,100.0,"Action,Animation,Sci-Fi",11,ghostintheshellthenewmovie,...,NaN,NaN,NaN,NaN,FUN,101000.0,NaN,101000.0,101000.0,Kazuya Nomura
1512,tt1368440,4.7,8480,Branded,Branded,2012,106.0,"Drama,Fantasy,Thriller",3,branded,...,NaN,NaN,NaN,NaN,RAtt.,354000.0,3400000.0,354000.0,354000.0,Jamie Bradshaw


In [32]:
total_movie_info.head()

,movie_id,averagerating,numvotes,primary_title,original_title,year,runtime_minutes,genres,ordering,title,...,release_date,production_budget,domestic_gross_movie_budgets,worldwide_gross_movie_budgets,studio,domestic_gross_movie_gross,foreign_gross_movie_gross,clean_domestic_gross,clean_worldwide_gross,primary_name
0,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",20,thelegendofhercules,...,"Jan 10, 2014",70000000.0,18848538.0,58953319.0,NaN,NaN,NaN,18848538.0,58953319.0,Renny Harlin
1,tt1171222,5.1,8296,Baggage Claim,Baggage Claim,2013,96.0,Comedy,5,baggageclaim,...,"Sep 27, 2013",8500000.0,21569509.0,22885836.0,NaN,NaN,NaN,21569509.0,22885836.0,David E. Talbert
2,tt1210166,7.6,326657,Moneyball,Moneyball,2011,133.0,"Biography,Drama,Sport",14,moneyball,...,"Sep 23, 2011",50000000.0,75605492.0,111300835.0,NaN,NaN,NaN,75605492.0,111300835.0,Bennett Miller
3,tt1212419,6.5,87288,Hereafter,Hereafter,2010,129.0,"Drama,Fantasy,Romance",4,hereafter,...,"Oct 15, 2010",50000000.0,32746941.0,108660270.0,NaN,NaN,NaN,32746941.0,108660270.0,Clint Eastwood
4,tt1232829,7.2,477771,21 Jump Street,21 Jump Street,2012,109.0,"Action,Comedy,Crime",26,21jumpstreet,...,"Mar 16, 2012",42000000.0,138447667.0,202812429.0,NaN,NaN,NaN,138447667.0,202812429.0,Christopher Miller


In [33]:
total_movie_info[["release_date"]]

,release_date
0,"Jan 10, 2014"
1,"Sep 27, 2013"
2,"Sep 23, 2011"
3,"Oct 15, 2010"
4,"Mar 16, 2012"
...,...
1532,NaN
1533,NaN
1534,NaN
1535,NaN


Dropped duplicates but kept movies that appeared twice. Some movies will have 2 different directors.

We are going to merge based the assumption that no director/directors released more than 1 movie on the exact same day

### Final Merged Table

Gives us income, budget, genre, date, title, director, rating

In [34]:
#Dropping NAN values
total_movie_info.dropna(subset=['primary_name', 'release_date'], inplace= True)
rt_movie_info.dropna(subset=['director', 'theater_date'], inplace= True)

#### Cleaning before the merge

In [35]:
#making the primary_name uniform in im.db directors table
total_movie_info["primary_name"] = total_movie_info["primary_name"].map(lambda x: re.sub(r'[^A-Za-z0-9]+', '', x).lower())
#making the director names uniform in rt_movie_info table
rt_movie_info["director"] = rt_movie_info["director"].map(lambda x: re.sub(r'[^A-Za-z0-9]+', '', x).lower())

In [36]:
#merging total_movie_info to rt_movie_info
total_movie_info_all= total_movie_info.merge(rt_movie_info, left_on=['primary_name','release_date'],right_on=['director','theater_date'], how="left")
total_movie_info_all

,movie_id,averagerating,numvotes,primary_title,original_title,year_x,runtime_minutes,genres,ordering,title,...,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio_y,year_y
0,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",20,thelegendofhercules,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt1171222,5.1,8296,Baggage Claim,Baggage Claim,2013,96.0,Comedy,5,baggageclaim,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt1210166,7.6,326657,Moneyball,Moneyball,2011,133.0,"Biography,Drama,Sport",14,moneyball,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt1212419,6.5,87288,Hereafter,Hereafter,2010,129.0,"Drama,Fantasy,Romance",4,hereafter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt1232829,7.2,477771,21 Jump Street,21 Jump Street,2012,109.0,"Action,Comedy,Crime",26,21jumpstreet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1180,tt2334871,4.5,29997,Snatched,Snatched,2017,90.0,"Action,Adventure,Comedy",12,snatched,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1181,tt2386490,7.6,60769,How to Train Your Dragon: The Hidden World,How to Train Your Dragon: The Hidden World,2019,104.0,"Action,Adventure,Animation",20,howtotrainyourdragonthehiddenworld,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1182,tt2704998,7.0,163279,Game Night,Game Night,2018,100.0,"Action,Comedy,Crime",15,gamenight,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1183,tt2704998,7.0,163279,Game Night,Game Night,2018,100.0,"Action,Comedy,Crime",15,gamenight,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
total_movie_info_all[["theater_date", "release_date", "director", "primary_name"]].head()

,theater_date,release_date,director,primary_name
0,NaN,"Jan 10, 2014",NaN,rennyharlin
1,NaN,"Sep 27, 2013",NaN,davidetalbert
2,NaN,"Sep 23, 2011",NaN,bennettmiller
3,NaN,"Oct 15, 2010",NaN,clinteastwood
4,NaN,"Mar 16, 2012",NaN,christophermiller


In [38]:
def dollar_to_float(column):
    new_col = column.replace('[$,()]', '', regex=True).astype(float)
    
    return new_col

def handle_NaN(col1, col2):
    new_col = col1.fillna(col2)
#     new_col = new_col.fillna(0)
total_movie_info_all["clean_domestic_gross"] = handle_NaN(total_movie_info_all["domestic_gross_movie_budgets"], total_movie_info_all["domestic_gross_movie_gross"])
total_movie_info_all["clean_worldwide_gross"] = handle_NaN(total_movie_info_all["worldwide_gross_movie_budgets"], total_movie_info_all["foreign_gross_movie_gross"])
total_movie_info_all["clean_domestic_gross"] = dollar_to_float(total_movie_info_all["clean_domestic_gross"])
total_movie_info_all["clean_worldwide_gross"] = dollar_to_float(total_movie_info_all["clean_worldwide_gross"])
total_movie_info_all["production_budget"] = dollar_to_float(total_movie_info_all["production_budget"])

In [39]:
#checking for dupicate movie_id's
total_movie_info_all[["box_office", "domestic_gross_movie_budgets"]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1185 entries, 0 to 1184
Data columns (total 2 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   box_office                    37 non-null     object 
 1   domestic_gross_movie_budgets  1185 non-null   float64
dtypes: float64(1), object(1)
memory usage: 27.8+ KB


In [40]:
total_movie_info_all[["box_office", "domestic_gross_movie_budgets"]].dropna()

,box_office,domestic_gross_movie_budgets
19,"80,574,010",80574010.0
20,"53,021,560",53032453.0
37,"102,981,571",103068524.0
76,"303,001,229",303003568.0
84,"26,973,524",27007844.0
85,"320,676",321910.0
89,"64,001,297",64003625.0
111,"21,379,315",21392758.0
117,"72,700,000",75624550.0
125,"98,000,000",98780042.0


In [41]:
# close the connection to the sql db
conn.close()